One of the main challenges when working with large language models (LLMs) is ensuring that the model output follows a specific structure. Since LLMs generate text based on probabilities, their responses may not always match the format or schema we expect, even when explicitly prompted.

We’ll use Ollama, which enables us to run local Llama models on our machine, combined with Langchain, to seamlessly integrate different steps of our solution

In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.5/599.5 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9

In [4]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.1 MB/s eta 0:00:00


In [5]:
!pip install langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 6.5 MB/s eta 0:00:00


In [6]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 21.7 MB/s eta 0:00:00


In [7]:
!pip install -qU langchain-ollama

In [8]:
!pip install ollama

In [9]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [10]:
from langchain import LLMChain, PromptTemplate
from langchain_huggingface import HuggingFacePipeline

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [1]:
from langchain_ollama import ChatOllama

In [2]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser, RetryOutputParser

In [4]:
from langchain_core.output_parsers.string import StrOutputParser
from typing import Optional

In [5]:
from pydantic import BaseModel, Field
from langchain.output_parsers import RetryOutputParser

In [6]:
from langchain_core.runnables import RunnableLambda, RunnableParallel
import subprocess

When working with large language models, it's important to note that not all models support structured outputs. For instance, llama3-chatqa:8b does not currently support adding another component for structured responses using with_structured_output method from langchain. However, the llama3.1 model supports structured outputs, making it a great choice for our tutorial. You can explore more models available on Ollama’s library: https://ollama.ai/library

Downloading the model will take a few minutes...

In [8]:
# now we pull of of the powerful llam3.1
!ollama pull llama3.1

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏ 426 KB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   1% ▕▏  38 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   1% ▕▏  67 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   

In [7]:
# I will use capture to remove the cell output, feel free to comment it to show all the progress bar
# we can now start ollama using ollama serve command
process = subprocess.Popen("ollama serve", shell=True)

#Initializing the model

Now we can use the freshly downloaded model. In our system there will be different components as follows:

1.   llm: which is the large language model used llama3.
2.   prompt template: this template contains
      * Basic instructions for the llm
      * Output schema that we want the model to follow
      * Finally the question that you want the model to answerList item
3.   the pydantic model: where you define the schema that you want your answer to follow
4.   the parser: two parsers will be used


      * The pydantic parser, which parses the llm output according to the pydantic model
      * The retry parser, which checks if the output follows the schema, if not it prompts the llm again but this time with the error, asking the model to retry using the schema.

In [9]:
llm = ChatOllama(
    model='llama3.1',
    temperature=0
)

In the template we will ask the model to give us the answer in a specif format, for instance we want to output to contain the question, the answer and rating for the confidence of the llm in the answer. we will also provide the format instruction and the query

In [10]:
template = """Based on the user question, provide an answer which consists of
the question, your answer to the question and a rating on how confidence you are.
{format_instructions}
Question: {query}
Response:"""

In [11]:
class Answer(BaseModel):

    question: str = Field(description="the question asked to the user")
    answer: str = Field(description="answer to the user question")
    rating: int = Field(description="How confident the answer is, from 1 to 10")

In [12]:
parser = PydanticOutputParser(pydantic_object=Answer)

In [13]:
retry_parser = RetryOutputParser.from_llm(parser=parser, llm=llm)

In [14]:
# now we will use a prompt template to be able to use the prompt above in a chain
# the prompt template uses the template and substitutes for the variables between curly brankets {}
# in the template above, it takes the format instructions from the pydantic parer
prompt = PromptTemplate(
    template=template,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

Now we can use two things:

1. structured_llm: which is done using with_structured_output, according to the documentation, link:https://python.langchain.com/docs/how_to/structured_output/

2. This is the easiest and most reliable way to get structured outputs. with_structured_output() is implemented for models that provide native APIs for structuring outputs, like tool/function calling or JSON mode, and makes use of these capabilities under the hood

In [15]:
structured_llm = llm.with_structured_output(Answer)

In [16]:
chain =  prompt | structured_llm

In [21]:
answer = chain.invoke("Who is ColdPlay?")

In [22]:
answer

Answer(question='Who is ColdPlay?', answer='ColdPlay is a British rock band formed in 1996.', rating=8)

In [23]:
print(f"question: {answer.question}")
print(f"answer: {answer.answer}")
print(f"confidence: {answer.rating}")

question: Who is ColdPlay?
answer: ColdPlay is a British rock band formed in 1996.
confidence: 8


But as mentioned earlier, it can still fail, so we can use the retry parser to reprompt the llm if it failed.

In [24]:
chain =  prompt | llm | StrOutputParser()

In [25]:
main_chain = (RunnableParallel(completion=chain, prompt_value=prompt) |
              RunnableLambda(lambda x: retry_parser.parse_with_prompt(**x)))

In [27]:
answer = main_chain.invoke("Who is ColdPlay?")

In [28]:
answer

Answer(question='Who is ColdPlay?', answer='Coldplay is a British rock band formed in 1996. The band consists of Chris Martin, Jonny Buckland, Guy Berryman, and Will Champion.', rating=9)

In [29]:
print(f"question: {answer.question}")
print(f"answer: {answer.answer}")
print(f"confidence: {answer.rating}")

question: Who is ColdPlay?
answer: Coldplay is a British rock band formed in 1996. The band consists of Chris Martin, Jonny Buckland, Guy Berryman, and Will Champion.
confidence: 9
